In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json
import warnings
import sys
import time
warnings.filterwarnings('ignore')

from tqdm import tqdm
from pathlib import Path
from collections import Counter, defaultdict
from PIL import Image

In [2]:
try:
    import google.colab
    from google.colab import drive
    !uv pip install anomalib
    !uv pip install open-clip-torch
    drive.mount('/content/drive', force_remount=True)
    PROJECT_ROOT = Path('/content/drive/Othercomputers/my_notebook/lion_final_pro_multimodal-anomaly-report-generation') # 본인 경로 수정: Mac/Window
except ImportError:
    PROJECT_ROOT = Path.cwd().parents[1]

os.chdir(PROJECT_ROOT) # 현재 경로 수정
print(f"Current working directory: {os.getcwd()}")

Using Python 3.12.12 environment at: /usr
Resolved 90 packages in 1.08s
Prepared 12 packages in 466ms
Installed 12 packages in 16ms
 + anomalib==2.2.0
 + freia==0.2
 + imagecodecs==2026.1.14
 + jsonargparse==4.45.0
 + kornia==0.8.2
 + kornia-rs==0.1.10
 + lightning==2.6.1
 + lightning-utilities==0.15.2
 + pytorch-lightning==2.6.1
 + rich-argparse==1.7.2
 + torchmetrics==1.8.2
 + typeshed-client==2.8.2
Using Python 3.12.12 environment at: /usr
Resolved 45 packages in 81ms
Prepared 2 packages in 29ms
Installed 2 packages in 4ms
 + ftfy==6.3.1
 + open-clip-torch==3.2.0
Mounted at /content/drive
Current working directory: /content/drive/Othercomputers/my_notebook/lion_final_pro_multimodal-anomaly-report-generation


In [3]:
# TODO: PATH 설정

# dataset path
DATA_ROOT = PROJECT_ROOT / "dataset" / "MMAD"
DOMAIN_JSON = DATA_ROOT / "domain_knowledge.json"
MMAD_JSON = DATA_ROOT / "mmad.json"
META_CSV = DATA_ROOT / "metadata.csv"

# config.yaml path
CONFIG_ROOT = PROJECT_ROOT / "configs"
RUNTIME_CONFIG_ROOT = CONFIG_ROOT / "runtime.yaml"
EVAL_CONFIG_ROOT = CONFIG_ROOT / "eval.yaml"

# output path
OUTPUT_ROOT = PROJECT_ROOT / "output"

# check (선택사항)
# print(f"Project Root: {PROJECT_ROOT}")
# print(f"Data Root: {DATA_ROOT}")
# print(f"Config Root: {CONFIG_ROOT}")

### MVTecAD Dataset

In [4]:
from src.utils import load_config, load_json, load_csv
from anomalib.data import MVTecAD
from anomalib.models import EfficientAd
from anomalib.engine import Engine

# TODO: runtime_config.yaml 확장/수정
runtime_config = load_config(RUNTIME_CONFIG_ROOT)
domain_json = load_json(DOMAIN_JSON)
mmad_json = load_json(MMAD_JSON)
meta_csv = load_csv(META_CSV)

# tqdm bar off
os.environ["TQDM_DISABLE"] = "1"
categories = [
    "bottle", "cable", "capsule", "carpet", "grid",
    "hazelnut", "leather", "metal_nut", "pill", "screw",
    "tile", "toothbrush", "transistor", "wood", "zipper"
]

In [6]:
# Train the model
for i, category in enumerate(categories, 1):
    print(f"\n[{i}/{len(categories)}] Testing: {category}...")

    start = time.time()

    datamodule = MVTecAD(
        root=DATA_ROOT / "MVTec-AD",
        category=category,
        train_batch_size=1
    )

    model = EfficientAd()
    engine = Engine(
        logger=False,
        enable_progress_bar=False,
        accelerator="auto", # cpu, mps
        devices=1,
        default_root_dir=OUTPUT_ROOT
    )

    engine.fit(datamodule=datamodule, model=model);

    elapsed = time.time() - start
    print(f"✓ [{i}/{len(categories)}] {category} 완료 ({elapsed:.1f}s)")

INFO:lightning_fabric.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:lightning_fabric.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:lightning_fabric.utilities.rank_zero:💡 Tip: For seamless cloud logging and experiment tracking, try installing [litlogger](https://pypi.org/project/litlogger/) to enable LitLogger, which logs metrics and artifacts automatically to the Lightning Experiments platform.



[1/15] Testing: bottle...


INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name           ┃ Type             ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ pre_processor  │ PreProcessor     │      0 │ train │     0 │
│ 1 │ post_processor │ PostProcessor    │      0 │ train │     0 │
│ 2 │ evaluator      │ Evaluator        │      0 │ train │     0 │
│ 3 │ model          │ EfficientAdModel │  8.1 M │ train │     0 │
└───┴────────────────┴──────────────────┴────────┴───────┴───────┘

Trainable params: 8.1 M                                                                                            
Non-trainable params: 0                                                                                            
Total params: 8.1 M                                                                                                
Total estimated model params size (MB): 32                                                                         
Modules in train mode: 48                                                                                          
Modules in eval mode: 7                                                                                            
Total FLOPs: 0

Calculate Validation Dataset Quantiles: 100%|██████████| 3/3 [00:03<00:00,  1.03s/it]
INFO:lightning_fabric.utilities.rank_zero:
Detected KeyboardInterrupt, attempting graceful shutdown ...
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/lightning/pytorch/trainer/call.py", line 49, in _call_and_handle_interrupt
    return trainer_fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/lightning/pytorch/trainer/trainer.py", line 630, in _fit_impl
    self._run(model, ckpt_path=ckpt_path, weights_only=weights_only)
  File "/usr/local/lib/python3.12/dist-packages/lightning/pytorch/trainer/trainer.py", line 1079, in _run
    results = self._run_stage()
              ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/lightning/pytorch/trainer/trainer.py", line 1123, in _run_stage
    self.fit_loop.run()
  File "/usr/local/lib/python3.12/dist-packages/lightning/pytorch/loops/fit_loop.py", line 217, in run
    self.advance()
  File "/usr/local/lib/python3.12/dist-packages/lightning/pytorch/loops/fit_loop.py", line 465, in advance
    self.epoch_loop.run(self._data_fetcher)
  File

TypeError: object of type 'NoneType' has no len()

In [ ]:
import logging
logging.disable(logging.WARNING)

# # tqdm bar on
# os.environ.pop("TQDM_DISABLE", None)

categories = [
    "bottle", "cable", "capsule", "carpet", "grid",
    "hazelnut", "leather", "metal_nut", "pill", "screw",
    "tile", "toothbrush", "transistor", "wood", "zipper"
]

all_predictions = {}
for i, category in enumerate(categories, 1):
    print(f"[{i}/{len(categories)}] Inference: {category}")
    # ckpt_path = OUTPUT_ROOT / category / "v0/weights/lightning/model.ckpt"
    ckpt_path = OUTPUT_ROOT / "EfficientAd" / "MVTecAD" / category / "v0/weights/lightning/model.ckpt"

    datamodule = MVTecAD(
          root=DATA_ROOT / "MVTec-AD",
          category=category
    )

    model = EfficientAd()
    engine = Engine(
        logger=False,
        enable_progress_bar=False,
        accelerator="auto",
        devices=1,
        default_root_dir=OUTPUT_ROOT
    )

    predictions = engine.predict(
        datamodule=datamodule,
        model=model,
        ckpt_path=ckpt_path,
    )

    all_predictions[category] = predictions
    print(f"✓ [{i}/{len(categories)}] {category} 완료\n")

In [ ]:
from src.visual.plot import kde_plot

y_true_list = []
y_score_list = []

for p in predictions:
    gt = p.gt_label.cpu().numpy() if hasattr(p.gt_label, 'cpu') else p.gt_label
    score = p.pred_score.cpu().numpy() if hasattr(p.pred_score, 'cpu') else p.pred_score
    y_true_list.append(gt)
    y_score_list.append(score)

y_true = np.concatenate(y_true_list)
y_score = np.concatenate(y_score_list)

normal_scores = y_score[y_true == 0]
anomaly_scores = y_score[y_true == 1]

scores_df = pd.DataFrame({
    'score': np.concatenate([normal_scores, anomaly_scores]),
    'label': ['Normal'] * len(normal_scores) + ['Anomaly'] * len(anomaly_scores)
})

kde_plot(
    scores_df,
    col='score',
    hue='label',
    palette=['steelblue', 'salmon'],
    title=f'{category}: Score Distribution',
    xlabel='Anomaly Score'
)

In [ ]:
import torch
from sklearn.metrics import (
    roc_auc_score,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    jaccard_score  # IoU
)

results = []
for category, preds in all_predictions.items():
    # Image-level
    y_true = np.concatenate([p.gt_label.cpu().numpy() for p in preds])
    y_score = np.concatenate([p.pred_score.cpu().numpy() for p in preds])
    y_pred = (y_score >= 0.5).astype(int)

    # Pixel-level
    gt_masks = torch.cat([p.gt_mask for p in preds]).int()
    pred_masks = torch.cat([(p.anomaly_map > 0.5).int() for p in preds])

    metrics = {
        "Category": category,
        "AUROC": round(roc_auc_score(y_true, y_score), 4),
        "Accuracy": round(accuracy_score(y_true, y_pred), 4),
        "Precision": round(precision_score(y_true, y_pred, zero_division=0), 4),
        "Recall": round(recall_score(y_true, y_pred, zero_division=0), 4),
        "F1": round(f1_score(y_true, y_pred, zero_division=0), 4),
        "Dice": round(
            f1_score(
                gt_masks.flatten().cpu().numpy(),
                pred_masks.flatten().cpu().numpy(),
                zero_division=0
            ),
            4
        ),
        "IoU": round(
            jaccard_score(
                gt_masks.flatten().numpy(),
                pred_masks.flatten().numpy(),
                average='binary',
                zero_division=0
            ),
            4
        ),
        "N_samples": len(y_true)
    }
    results.append(metrics)

metrics_df = pd.DataFrame(results).set_index("Category")
avg_row = metrics_df.drop(columns=['N_samples']).mean().round(4)
avg_row['N_samples'] = metrics_df['N_samples'].sum()
metrics_df.loc['Average'] = avg_row

metrics_df

In [ ]:
from src.visual.plot import heatmap_plot

metrics_trans = metrics_df.drop('Average').drop(columns='N_samples')
heatmap_plot(
    metrics_trans,
    figsize=(10, 10),
    cmap='RdYlGn',
    annot=True,
    fmt='.4f',
    linewidths=0.5,
    title='PatchCore Performance by Category',
    rotation_x=45,
    rotation_y=0
)

In [ ]:
from src.visual.plot import anomaly_grid_from_dir

for category in categories:
    OUTPUT_IMG_ROOT = OUTPUT_ROOT / "Patchcore" / "MVTecAD" / category / "latest" / "images"
    print(f"{category}")
    anomaly_grid_from_dir(OUTPUT_IMG_ROOT, n_samples=1, n_cols=1)